# aT 농산품 예측 base line

&nbsp;

In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import warnings
from glob import glob
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from pandasql import sqldf
import os


# 경고 끄기
warnings.filterwarnings(action='ignore')

&nbsp;

## 전처리 Class

In [2]:
class preprocessing_data(object):

    """
    도매, 소매, 수입수출, 도매경락, 주산지 데이터 전처리용 class
    중간결과물 저장 check parameter을 통해 지정, 중간결과물 저장 없이 사용은 check = 0
    """

    def __init__(self,dir):
        """
        전체 데이터에서 해당하는 domae,imexport,pummok,somae,weather 별 분리
        """
        self.data_list = glob(dir)
        self.domae = []
        self.imexport = []
        self.pummok = []
        self.somae = []
        self.weather = []


        for i in self.data_list:
            if 'domae' in i:
                self.domae.append(i)
            if 'imexport' in i:
                self.imexport.append(i)
            if 'pummok' in i.split('/')[-1]:
                self.pummok.append(i)
            if 'somae' in i:
                self.somae.append(i)
            if 'weather' in i:
                self.weather.append(i)


    def add_pummock(self,check=0):

        """
        check = 중간 산출물을 저장하고 싶다면 check 을 0 이외의 숫자로
        pummock의 데이터를 가져와 '해당일자_전체거래물량', '하위가격 평균가', '상위가격 평균가', '하위가격 거래물량', '상위가격 거래물량' 의 파생변수를 생성하는 단계
        """

        for num in tqdm(self.pummok):
            ddf = pd.read_csv(num)  # pummock의 csv 읽어오기
            name = num.split('/')[-1] # 전체 정제한 데이터를 담을 변수 이름

            sep2 = sqldf(f"select *, sum(거래량) as '해당일자_전체거래물량(kg)' from ddf group by datadate")
            # sql 문법을 이용해 '해당일자_전체거래물량' 계산

            height_set = []
            low_set = []
            height_volume_set = []
            low_volume_set = []

            for i in sep2['datadate']:

                """
                sep2는 group by를 통해 각 일자가 합쳐진 상태 예를 들어 '201703' 이 5개 이렇게 있을때 sep2는 group 시켜서 '해당일자_전체거래물량'을 계산
                이후 sep2 기준 20170101 and 20220630 사이의 날짜들에 해당하는 각 '201703' 마다 '해당일자_전체평균가격' 보다 큰지 아니면 작은지 판단
                위 과정을 통해 '하위가격 평균가', '상위가격 평균가', '하위가격 거래물량', '상위가격 거래물량' 변수 생성
                """

                new_list = ddf.loc[[d for d, x in enumerate(ddf['datadate']) if x == i]]
                set_price = sep2.loc[list(sep2['datadate']).index(i)]['해당일자_전체평균가격(원)']

                sum_he_as = sum(new_list['거래대금(원)'].iloc[n] for n, z in enumerate(new_list['단가(원)']) if z >= set_price)
                sum_he_vo = sum(new_list['거래량'].iloc[n] for n, z in enumerate(new_list['단가(원)']) if z >= set_price)

                sum_lo_as = sum(new_list['거래대금(원)'].iloc[n] for n, z in enumerate(new_list['단가(원)']) if z < set_price)
                sum_lo_vo = sum(new_list['거래량'].iloc[n] for n, z in enumerate(new_list['단가(원)']) if z < set_price)

                if sum_lo_vo != 0:
                    low_set.append(sum_lo_as / sum_lo_vo)
                    low_volume_set.append(sum_lo_vo)
                else:
                    low_set.append(np.nan)
                    low_volume_set.append(np.nan)

                if sum_he_vo != 0:
                    height_set.append(sum_he_as / sum_he_vo)
                    height_volume_set.append(sum_he_vo)
                else:
                    height_set.append(np.nan)
                    height_volume_set.append(np.nan)

            sep2['하위가격 평균가(원)'] = low_set
            sep2['상위가격 평균가(원)'] = height_set

            sep2['하위가격 거래물량(kg)'] = low_volume_set
            sep2['상위가격 거래물량(kg)'] = height_volume_set


            globals()[f'df_{name.split("_")[1].split(".")[0]}'] = sep2.copy()


            # 중간 산출물 저장
            if check != 0:
                if os.path.exists(f'./data') == False:
                    os.mkdir(f'./data')

                if os.path.exists(f'./data/품목') == False:
                    os.mkdir(f'./data/품목')

                sep2.to_csv(f'./data/품목/{name}', index=False)




    def add_dosomae(self, option=1, check=0):

        """
        check = 중간 산출물을 저장하고 싶다면 check 을 0 이외의 숫자로
        domae, somae 데이터를 가져와서 정제하는 단계
        option parameter을 통한 도매, 소매 선택
        """

        if option == 1:
            df = self.domae
            text = '도매'
        else:
            df = self.somae
            text = '소매'

        for i in tqdm(df):
            test = pd.read_csv(i)
            name = i.split('/')[-1]

            sep = test.loc[(test['등급명'] == '상품') | (test['등급명'] == 'S과')]  # 모든 상품에 대해서 수행하지 않고 GRAD_NM이 '상품', 'S과' 만 해당하는 품목 가져옴
            sep = sep[['datadate', '등급명', '조사단위(kg)', '가격(원)']]

            sep.rename(columns={"가격(원)": "가격"}, inplace=True)

            sep2 = sqldf(
                f"select datadate, max(가격) as '일자별_{text}가격_최대(원)', avg(가격) as '일자별_{text}가격_평균(원)', min(가격) as '일자별_{text}가격_최소(원)' from sep group by datadate")

            globals()[f'df_{name.split("_")[1].split(".")[0]}'] = globals()[f'df_{name.split("_")[1].split(".")[0]}'].merge(sep2, how='left')

            # 중간 산출물 저장
            if check != 0:
                if os.path.exists(f'./data') == False:
                    os.mkdir(f'./data')

                if os.path.exists(f'./data/{text}') == False:
                    os.mkdir(f'./data/{text}')

                sep2.to_csv(f'./data/{text}/{name}', index=False)

    def add_imexport(self,check=0):
        """
        check = 중간 산출물을 저장하고 싶다면 check 을 0 이외의 숫자로
        imexport 데이터 관련 정제, imexport 데이터는 월별 수입수출 데이터임으로 해당 월에 같은 값을 넣어주고 없는 것에는 np.nan
        해당 품목에 대한 imexport 데이터가 없는 경우 np.nan으로 대체, 모든 품목의 데이터가 동일한 컬럼수를 가지기 위해 수행
        """

        imex_cd = [i.split('_')[-1].split('.')[0] for i in self.imexport]

        for i in tqdm(range(len(self.pummok))):

            cd_number = self.pummok[i].split('_')[-1].split('.')[0]
            file_name = 'imexport_' + self.pummok[i].split('pummok_')[1]


            if cd_number in imex_cd:
                test4 = pd.read_csv(self.imexport[imex_cd.index(cd_number)])

                new_exim1 = []
                new_exim2 = []
                new_exim3 = []
                new_exim4 = []
                new_exim5 = []

                for j in globals()[f'df_{cd_number}']['datadate']:
                    target = j//100

                    try:
                        number = list(test4['datadate']).index(target)


                        new_exim1.append(test4['수출중량(kg)'].iloc[number])
                        new_exim2.append(test4['수출금액(달러)'].iloc[number])
                        new_exim3.append(test4['수입중량(kg)'].iloc[number])
                        new_exim4.append(test4['수입금액(달러)'].iloc[number])
                        new_exim5.append(test4['무역수지(달러)'].iloc[number])
                    except:
                        new_exim1.append(np.nan)
                        new_exim2.append(np.nan)
                        new_exim3.append(np.nan)
                        new_exim4.append(np.nan)
                        new_exim5.append(np.nan)

                df2 = pd.DataFrame()
                df2['수출중량(kg)'] = new_exim1
                df2['수출금액(달러)'] = new_exim2
                df2['수입중량(kg)'] = new_exim3
                df2['수입금액(달러)'] = new_exim4
                df2['무역수지(달러)'] = new_exim5

                globals()[f'df_{cd_number}'] = pd.concat([globals()[f'df_{cd_number}'], df2],axis=1)

            else:
                df2 = pd.DataFrame()
                df2['수출중량(kg)'] = np.nan
                df2['수출금액(달러)'] = np.nan
                df2['수입중량(kg)'] = np.nan
                df2['수입금액(달러)'] = np.nan
                df2['무역수지(달러)'] = np.nan

                globals()[f'df_{cd_number}'] = pd.concat([globals()[f'df_{cd_number}'], df2], axis=1)


            if check != 0:
                if os.path.exists(f'./data') == False:
                    os.mkdir(f'./data')

                if os.path.exists(f'./data/수출입') == False:
                    os.mkdir(f'./data/수출입')

                df2.to_csv(f'./data/수출입/{file_name}', index=False)

    def add_weather(self, check=0):

        """
        check = 중간 산출물을 저장하고 싶다면 check 을 0 이외의 숫자로
        weather 품목별 주산지 데이터를 가져와 합치는 함수, 일부 품목의 주산지가 3개가 아닌 것에 대해서는 np.nan 값으로 합쳐줌
        """

        for i in tqdm(self.pummok):
            name = i.split('_')[-1].split('.')[0]
            check_file = [j for j in self.weather if j.split('_')[-2] == name]


            df = pd.DataFrame()
            for d, j in enumerate(check_file):
                weather_df = pd.read_csv(j)
                new_exim1, new_exim2, new_exim3, new_exim4, new_exim5, new_exim6 = [], [], [], [], [], []


                for k in globals()[f'df_{name}']['datadate']:
                    try:
                        number = list(weather_df['datadate']).index(k)

                        new_exim1.append(weather_df['초기온도(℃)'].iloc[number])
                        new_exim2.append(weather_df['최대온도(℃)'].iloc[number])
                        new_exim3.append(weather_df['최저온도(℃)'].iloc[number])
                        new_exim4.append(weather_df['평균온도(℃)'].iloc[number])
                        new_exim5.append(weather_df['강수량(ml)'].iloc[number])
                        new_exim6.append(weather_df['습도(%)'].iloc[number])
                    except:
                        new_exim1.append(np.nan)
                        new_exim2.append(np.nan)
                        new_exim3.append(np.nan)
                        new_exim4.append(np.nan)
                        new_exim5.append(np.nan)
                        new_exim6.append(np.nan)


                df[f'주산지_{d}_초기온도(℃)'] = new_exim1
                df[f'주산지_{d}_최대온도(℃)'] = new_exim2
                df[f'주산지_{d}_최저온도(℃)'] = new_exim3
                df[f'주산지_{d}_평균온도(℃)'] = new_exim4
                df[f'주산지_{d}_강수량(ml)'] = new_exim5
                df[f'주산지_{d}_습도(%)'] = new_exim6

            if len(check_file) < 3:
                df[f'주산지_2_초기온도(℃)'] = np.nan
                df[f'주산지_2_최대온도(℃)'] = np.nan
                df[f'주산지_2_최저온도(℃)'] = np.nan
                df[f'주산지_2_평균온도(℃)'] = np.nan
                df[f'주산지_2_강수량(ml)'] = np.nan
                df[f'주산지_2_습도(%)'] = np.nan

            globals()[f'df_{name}'] = pd.concat([globals()[f'df_{name}'], df], axis=1)

            if check !=0:
                if os.path.exists(f'./data') == False:
                    os.mkdir(f'./data')

                if os.path.exists(f'./data/주산지') == False:
                    os.mkdir(f'./data/주산지')

                df.to_csv(f'./data/주산지/weather_{name}.csv', index=False)

    def add_categorical(self, out_dir, data_type="train", check=0):

        """
        check = 중간 산출물을 저장하고 싶다면 check 을 0 이외의 숫자로
        일자별 정보를 넣어주는 함수, 월별, 상순, 하순, 중순 을 원핫 인코딩을 통해 데이터로 넣어주는 함수
        모델이 각 행마다의 정보에서 몇월인지 상순인지 하순인지 파악하며 훈련시키기 위한 변수
        """

        for i in tqdm(self.pummok):
            name = i.split('_')[-1].split('.')[0]

            day_set = []
            month_set = []

            for k in globals()[f'df_{name}']['datadate']:
                day = k % 100
                month = k % 10000 // 100

                if day <= 10:
                    day_set.append('초순')
                elif (day > 10) and (day <= 20):
                    day_set.append('중순')
                else:
                    day_set.append('하순')

                month_set.append(f'{month}월')

            globals()[f'df_{name}']['일자구분'] = day_set
            globals()[f'df_{name}']['월구분'] = month_set

            globals()[f'df_{name}'] = pd.get_dummies(globals()[f'df_{name}'], columns=['일자구분', '월구분'])

            if check !=0:
                if os.path.exists(f'./data') == False:
                    os.mkdir(f'./data')

                if data_type != "train":
                    if os.path.exists(f'./data/{data_type}') == False:
                        os.mkdir(f"./data/{data_type}")
                    if os.path.exists(f'./data/{data_type}/{out_dir}') == False:
                        os.mkdir(f'./data/{data_type}/{out_dir}')
                    globals()[f'df_{name}'].to_csv(f'./data/{data_type}/{out_dir}/{data_type}_{name}.csv', index=False)
                else:
                    if os.path.exists(f'./data/{out_dir}') == False:
                        os.mkdir(f'./data/{out_dir}')
                    globals()[f'df_{name}'].to_csv(f'./data/{out_dir}/{data_type}_{name}.csv', index=False)

&nbsp;

## 훈련 데이터 전처리 및 저장 (중간저장 X, 최종저장 O) - train

In [9]:
data = preprocessing_data('./aT_train_raw/*.csv')
data.add_pummock()
data.add_dosomae()
data.add_dosomae(option=2)
data.add_imexport()
data.add_weather()
data.add_categorical('train', data_type="train" ,check=1)

0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]


&nbsp;

## 검증 데이터셋 전처리 및 저장 (중간저장 X, 최종저장 O) - test

In [10]:
for i in range(10):
    data = preprocessing_data(f'./aT_test_raw/sep_{i}/*.csv')
    data.add_pummock()
    data.add_dosomae()
    data.add_dosomae(option=2)
    data.add_imexport()
    data.add_weather()
    data.add_categorical(f'set_{i}', data_type="test", check=1)

0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, 

&nbsp;

## 입력 shape 및 형태 정의 함수

In [11]:
def make_Tensor(array):
    return tf.convert_to_tensor(array, dtype=tf.float32)

def astype_data(data):
    df = data.astype(np.float32)
    return make_Tensor(df)

&nbsp;

## Transformer 정의

- encoder

In [12]:
def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):

    x = layers.LayerNormalization(epsilon=1e-6)(inputs)
    x = layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout
    )(x, x)
    x = layers.Dropout(dropout)(x)
    res = x + inputs

    x = layers.LayerNormalization(epsilon=1e-6)(res)
    x = layers.Conv1D(filters=ff_dim, kernel_size=1, activation="relu")(x)
    x = layers.Dropout(dropout)(x)
    x = layers.Conv1D(filters=inputs.shape[-1], kernel_size=1)(x)
    return x + res

- build

In [13]:
def build_model(input_shape, head_size, num_heads, ff_dim, num_transformer_blocks, mlp_units, dropout=0, mlp_dropout=0):
    inputs = keras.Input(shape=input_shape)
    x = inputs
    for _ in range(num_transformer_blocks):
        x = transformer_encoder(x, head_size, num_heads, ff_dim, dropout)

    x = layers.GlobalAveragePooling1D(data_format="channels_first")(x)
    for dim in mlp_units:
        x = layers.Dense(dim, activation="relu")(x)
        x = layers.Dropout(mlp_dropout)(x)
    outputs = layers.Dense(28)(x) # 4주 예측
    return keras.Model(inputs, outputs)

&nbsp;

## keras eraly stop, chekpoint 정의

In [14]:
def call_back_set(name, epoch, batch_size):
    early_stopping = EarlyStopping(monitor='val_loss', patience=100)

    if os.path.exists(f'./check') == False:
        os.mkdir(f'./check')

    filename = f'./check/{name}-{epoch}-{batch_size}.h5'

    checkpoint = ModelCheckpoint(filename,
                                 monitor='val_loss',
                                 verbose=1,
                                 save_best_only=True,
                                 save_weights_only=True,
                                 mode='auto'
                                 )
    return [early_stopping, checkpoint]

&nbsp;

## Model 훈련 함수

In [15]:
def train(x_train, y_train, x_val, y_val, name, epoch, batch_size, learning_rate = 0.001, verbose = 1):


    model = build_model(
    x_train.shape[1:],
    head_size=256,
    num_heads=4,
    ff_dim=4,
    num_transformer_blocks=4,
    mlp_units=[128],
    mlp_dropout=0.4,
    dropout=0.25,
    )

    model.compile(
        loss="mean_squared_error",
        optimizer=keras.optimizers.Adam(learning_rate=learning_rate)
    )


    # Train the model
    with tf.device('/device:GPU:0'):
        history1 = model.fit(
            x_train, y_train,
            epochs = epoch,
            steps_per_epoch=len(x_train) / batch_size,
            batch_size=batch_size,
            validation_data=(x_val, y_val),
            validation_steps=len(x_val) / batch_size,
            shuffle=False,
            callbacks=call_back_set(name, epoch, batch_size),
            verbose=verbose)

    return model

&nbsp;

## 시점 윈도우 생성 함수

In [16]:
def time_window(df, t, t_sep):
    seq_len = t
    seqence_length = seq_len + t_sep

    result = []
    for index in tqdm(range(len(df) - seqence_length)):
        result.append(df[index: index + seqence_length].values)

    return np.array(result)

&nbsp;

## 데이터 불러오기 및 parameter 설정

In [17]:
data_list = glob('./data//train/*.csv')
epoch = 1000
batch = 15
tr_del_list = ['단가(원)', '거래량', '거래대금(원)', '경매건수', '도매시장코드', '도매법인코드', '산지코드 '] # train 에서 사용하지 않는 열
ts_del_list = ['단가(원)', '거래량', '거래대금(원)', '경매건수', '도매시장코드', '도매법인코드', '산지코드 ', '해당일자_전체평균가격(원)'] # test 에서 사용하지 않는 열
check_col = ['일자구분_중순', '일자구분_초순', '일자구분_하순','월구분_10월', '월구분_11월', '월구분_12월', '월구분_1월', '월구분_2월', '월구분_3월', 
             '월구분_4월','월구분_5월', '월구분_6월', '월구분_7월', '월구분_8월', '월구분_9월'] # 열 개수 맞추기

&nbsp;

## Train 과정

In [18]:
for i in tqdm(data_list):
    df_number = i.split("_")[-1].split(".")[0]
    df = pd.read_csv(i)

    for j in df.columns:
        df[j] = df[j].replace({' ': np.nan})

    # 사용할 열 선택 및 index 설정
    df.drop(tr_del_list, axis=1, inplace=True)
    df.set_index('datadate', drop=True, inplace=True)

    # nan 처리
    df = df.fillna(0)

    # 변수와 타겟 분리
    x, y = df[[i for i in df.columns if i != '해당일자_전체평균가격(원)']], df['해당일자_전체평균가격(원)']

    # 2주 입력을 통한 이후 4주 예측을 위해 y의 첫 14일을 제외
    y = y[14:]

    # time series window 생성
    data_x = time_window(x, 13, 1)
    data_y = time_window(y, 27, 1)

    # y의 길이와 같은 길이로 설정
    xdata = data_x[:len(data_y)]
    ydata = data_y

    # train, validation 분리 (8 : 2)
    x_train, x_val, y_train, y_val = train_test_split(xdata, ydata, test_size=0.2, shuffle=False, random_state=119)

    # transformer 모델 훈련
    transformer = train(astype_data(x_train), y_train, astype_data(x_val), y_val, f'transformer-{df_number}', epoch,
                        batch)
    transformer.load_weights(f'./check/transformer-{df_number}-{epoch}-{batch}.h5')

    if os.path.exists(f'./model') == False:
        os.mkdir(f'./model')

    # 모델 저장
    transformer.save(f'./model/transformer-{df_number}-{epoch}-{batch}.h5')

0it [00:00, ?it/s]


&nbsp;

## Test 과정

In [19]:
zero_csv = [0 for i in range(14)]  # 시점이 비어있는 데이터 0으로 채우기 위한 변수

for i in tqdm(range(10)):
    data_list = glob(f'./data/test/set_{i}/*.csv')

    for idx,j in enumerate(data_list):
        df = pd.read_csv(j)

        if len(df) == 0:
            df['zero_non'] = zero_csv
            df = df.fillna(0)
            df.drop('zero_non', axis=1, inplace=True)


        file_number = j.split('test_')[1].split('.')[0]

        # 사용할 열 선택, index 설정
        df.drop(ts_del_list, axis=1, inplace=True)
        df.set_index('datadate', drop=True, inplace=True)

        # train input 과 형상 맞추기
        add_col = [i for i in check_col if i not in df.columns]

        for a in add_col:
            df[a] = 0

        # ' ' -> nan 으로 변경
        for a in df.columns:
            df[a] = df[a].replace({' ': np.nan})

        # nan 처리
        df = df.fillna(0)

        # x_test  생성
        df_test = astype_data(df.values.reshape(1, df.values.shape[0], df.values.shape[1]))


        # model test
        if os.path.exists('./model_output') == False:
            os.mkdir('./model_output')

        if os.path.exists(f'./model_output/set_{i}') == False:
            os.mkdir(f'./model_output/set_{i}')

        # 해당하는 모델 불러오기
        model_test = tf.keras.models.load_model(f'./model/transformer-{file_number}-{epoch}-{batch}.h5')
        pred = transformer.predict(df_test)


        # 결과 저장
        save_df = pd.DataFrame(pred).T
        save_df.to_csv(f'./model_output/set_{i}/predict_{file_number}.csv', index=False)

100%|██████████| 10/10 [00:00<00:00, 6435.94it/s]


&nbsp;

## 정답 제출 파일생성

In [20]:
for k in tqdm(range(10)):

  globals()[f'set_df_{k}'] = pd.DataFrame()
  answer_df_list = glob(f'./model_output/set_{k}/*.csv') # 예측한 결과 불러오기
  pum_list = glob(f'./aT_test_raw/sep_{k}/*.csv') # 기존 test input 불러오기
  pummok = [a for a in pum_list if 'pummok' in a.split('/')[-1]]

  for i in answer_df_list:
    df = pd.read_csv(i)
    number = i.split('_')[-1].split('.')[0]

    base_number = 0
    for p in pummok:
      if number == p.split('_')[-1].split('.')[0]:
        pum_df = pd.read_csv(p)

        if len(pum_df) != 0:
           base_number = pum_df.iloc[len(pum_df)-1]['해당일자_전체평균가격(원)']  # 기존 각 sep 마다 test input의 마지막 target 값 가져오기 (변동률 계산을 위해)
        else:
          base_number = np.nan

    globals()[f'set_df_{k}'][f'품목{number}']  = [base_number] + list(df[df.columns[-1]].values) # 각 품목당 순서를 t, t+1 ... t+28 로 변경

  globals()[f'set_df_{k}'] = globals()[f'set_df_{k}'][[f'품목{col}' for col in range(37)]] # 열 순서를 품목0 ~ 품목36 으로 변경

  0%|          | 0/10 [00:00<?, ?it/s]


KeyError: "None of [Index(['품목0', '품목1', '품목2', '품목3', '품목4', '품목5', '품목6', '품목7', '품목8', '품목9',\n       '품목10', '품목11', '품목12', '품목13', '품목14', '품목15', '품목16', '품목17', '품목18',\n       '품목19', '품목20', '품목21', '품목22', '품목23', '품목24', '품목25', '품목26', '품목27',\n       '품목28', '품목29', '품목30', '품목31', '품목32', '품목33', '품목34', '품목35', '품목36'],\n      dtype='object')] are in the [columns]"

- 변동률 계산을 위한 t, t+1 ... t+28 설정

In [ ]:
set_df_0

: 

- 변동률 계산 

In [ ]:
date = [f'd+{i}' for i in range(1,15)] + ['d+22 ~ 28 평균']


for k in range(10):
  globals()[f'answer_df_{k}'] = pd.DataFrame()
  for c in globals()[f'set_df_{k}'].columns:
    base_d = globals()[f'set_df_{k}'][c][0] # 변동률 기준 t 값

    ans_1_14 = []
    for i in range(14):
      ans_1_14.append((globals()[f'set_df_{k}'][c].iloc[i+1]- base_d)/base_d)  # t+1 ~ t+14 까지는 (t+n - t)/t 로 계산

    ans_22_28 = (globals()[f'set_df_{k}'][c][22:29].mean() - base_d)/base_d # t+22 ~ t+28은 np.mean(t+22 ~ t+28) - t / t

    globals()[f'answer_df_{k}'][f'{c} 변동률'] = ans_1_14 + [ans_22_28]
  
  globals()[f'answer_df_{k}']['Set'] = k # set 번호 설정
  globals()[f'answer_df_{k}']['일자'] = date # 일자 설정

: 

- sep 0  ~ sep 9 까지 합치기

In [ ]:
# 위에서 계산된 변동률 들을 합쳐주는 과정

all_df =pd.DataFrame()
for i in range(10):
  if i== 0 :
    all_df = pd.concat([all_df, globals()[f'answer_df_{i}']],axis=1)
  else:
    all_df = pd.concat([all_df, globals()[f'answer_df_{i}']])


all_df = all_df[['Set','일자'] + list(all_df.columns[:-2])]
all_df.reset_index(drop=True, inplace=True)

: 

- 정답 양식으로 변경

In [ ]:
# set, 일자 기억하기위해 따로 저장

re_set = list(all_df['Set'])
re_date = list(all_df['일자'])


# 정답 양식 불러오기
out_ans = pd.read_csv('./answer_example.csv')

# 두 dataframe 합치기 (nan + 숫자 = nan 이용)
submit_df = all_df + out_ans

submit_df['Set'] = re_set
submit_df['일자'] = re_date


# 최종 저장
submit_df.to_csv('./submit.csv',index=False)

: 

- 계산된 변동률 결과물

In [ ]:
all_df

: 

- 제출 양식

In [ ]:
out_ans

: 

- 제출 양식 반영한 최종 결과물 (**실 제출용**)

In [ ]:
submit_df

: 